In [1]:
# import libraries
import pandas as pd
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
import numpy as np # library to handle data in a vectorized manner

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

## 1. Download and Preprocess Dataset

Scraping Wikipedia page in order to obtain the data that is in the table of administrative divisions of Beijing and read it into a pandas dataframe:

In [2]:
neighborhoods = pd.read_html('https://en.wikipedia.org/wiki/List_of_administrative_divisions_of_Beijing')[4]

city="Beijing"
nation="China"

We rename the column Name to Borough and display the dataframe.

In [3]:
neighborhoods=neighborhoods.rename(columns={"Name": "Borough"})
neighborhoods.head()

,Unnamed: 0,Borough,Chinese,Hanyu Pinyin,Division code[1],Division code[1].1,Area (km²),Population (2010 census)[2],Density (/km²)
0,NaN,Dongcheng District(City seat),东城区,Dōngchéng Qū,110101,DCQ,40.6,919000,22635
1,NaN,Xicheng District,西城区,Xīchéng Qū,110102,XCQ,46.5,1243000,26731
2,NaN,Chaoyang District,朝阳区,Cháoyáng Qū,110105,CYQ,470.8,3545000,7530
3,NaN,Fengtai District,丰台区,Fēngtái Qū,110106,FTQ,304.2,2112000,6943
4,NaN,Shijingshan District,石景山区,Shíjǐngshān Qū,110107,SJS,89.8,616000,6860


In [4]:
print('number of rows in the dataframe:', neighborhoods.shape[0])

number of rows in the dataframe: 16


Read geographical coordinates of each district:

In [5]:
from geopy.geocoders import Nominatim
import geocoder
#Defining a function to get the location of the localities
def get_location(localities):
    city_str = '{}, ' + city
    g = geocoder.arcgis(city_str.format(localities))
    get_latlng = g.latlng
    return get_latlng

#Creating an empty list
co_ordinates = []
#Getting the co-ordinates of each locality using the function defined above
for i in neighborhoods["Borough"].tolist():
    print(i, get_location(i))
    co_ordinates.append(get_location(i))

#Add coordinates to dataframe
neighborhoods['Latitude'] = [la for [la, lo] in co_ordinates]
neighborhoods['Longitude'] = [lo for [la, lo] in co_ordinates]


Dongcheng District(City seat) [39.93264000000005, 116.41834000000006]
Xicheng District [39.933170000000075, 116.37337000000002]
Chaoyang District [39.954620000000034, 116.50290000000007]
Fengtai District [39.847570000000076, 116.27227000000005]
Shijingshan District [39.91124000000008, 116.18766000000005]
Haidian District [39.99064000000004, 116.28868000000011]
Mentougou District [39.93819000000008, 116.09307000000001]
Fangshan District [39.698330000000055, 115.99250000000006]
Tongzhou District(City seat) [39.903950000000066, 116.66183000000001]
Shunyi District [40.12175000000008, 116.64783000000011]
Changping District [40.216120000000046, 116.23471000000006]
Daxing District [39.74025000000006, 116.32693000000006]
Huairou District [40.32353000000006, 116.63076000000001]
Pinggu District [40.14554000000004, 117.10948000000008]
Miyun District [40.37587000000008, 116.8419100000001]
Yanqing District [40.460930000000076, 115.9715000000001]


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>explorer</em>, as shown below.


In [6]:
geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of ' + city + ' are {}, {}.'.format(latitude, longitude))

print(neighborhoods.head())

The geograpical coordinate of Beijing are 39.9020803, 116.7185213.
   Unnamed: 0                        Borough Chinese    Hanyu Pinyin  \
0         NaN  Dongcheng District(City seat)     东城区    Dōngchéng Qū   
1         NaN               Xicheng District     西城区      Xīchéng Qū   
2         NaN              Chaoyang District     朝阳区     Cháoyáng Qū   
3         NaN               Fengtai District     丰台区      Fēngtái Qū   
4         NaN           Shijingshan District    石景山区  Shíjǐngshān Qū   

   Division code[1] Division code[1].1  Area (km²)  \
0            110101                DCQ        40.6   
1            110102                XCQ        46.5   
2            110105                CYQ       470.8   
3            110106                FTQ       304.2   
4            110107                SJS        89.8   

   Population (2010 census)[2]  Density (/km²)  Latitude  Longitude  
0                       919000           22635  39.93264  116.41834  
1                      1243000     

#### Create a map of Berlin with neighborhoods superimposed on top.


In [7]:
# create map of Beijing using latitude and longitude values
map_city = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough']):
    label = borough
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_city) 
    
map_city

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.


## 2. Explore Neighborhoods in Berlin

#### Define Foursquare Credentials and Version

In [8]:
CLIENT_ID = # your Foursquare ID
CLIENT_SECRET = # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BGPSQFDUTJNLEDRMY2N2AJVOYBD3BEELQDGHHFB3ZV0WZGI4
CLIENT_SECRET:LB0VHC4WCZ5OUJ2OHTIOCQI112AWCVCKCQW1TN0YVIEI5KFM


#### Let's create a function to repeat the same process to all the neighborhoods in Berlin

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
city_venues = getNearbyVenues(neighborhoods['Borough'], neighborhoods['Latitude'], neighborhoods['Longitude'])

Dongcheng District(City seat)
Xicheng District
Chaoyang District
Fengtai District
Shijingshan District
Haidian District
Mentougou District
Fangshan District
Tongzhou District(City seat)
Shunyi District
Changping District
Daxing District
Huairou District
Pinggu District
Miyun District
Yanqing District


Filter venues (only restaurants and pizza places) and check the size of the resulting dataframe.

In [11]:
city_venues.shape
city_venues["Venue Category"].values
city_venues = city_venues[(city_venues["Venue Category"].str.contains("Restaurant")) | (city_venues["Venue Category"].str.contains("Pizza"))]
city_venues.shape

(169, 7)

Let's check how many venues were returned for each neighborhood


In [12]:
city_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Chaoyang District,35,35,35,35,35,35
Dongcheng District(City seat),28,28,28,28,28,28
Fangshan District,3,3,3,3,3,3
Fengtai District,3,3,3,3,3,3
Haidian District,50,50,50,50,50,50
Huairou District,2,2,2,2,2,2
Mentougou District,2,2,2,2,2,2
Miyun District,3,3,3,3,3,3
Pinggu District,4,4,4,4,4,4


#### Let's find out how many unique categories can be curated from all the returned venues

In [13]:
print('There are {} uniques categories.'.format(len(city_venues['Venue Category'].unique())))

There are 35 uniques categories.


## 3. Analyze Each Neighborhood

In [14]:
# one hot encoding
city_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
city_onehot['Neighborhood'] = city_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [city_onehot.columns[-1]] + list(city_onehot.columns[:-1])
city_onehot = city_onehot[fixed_columns]

city_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,Australian Restaurant,Beijing Restaurant,Cantonese Restaurant,Chinese Aristocrat Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,...,Restaurant,Scandinavian Restaurant,South American Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Xinjiang Restaurant,Yunnan Restaurant,Zhejiang Restaurant
5,Dongcheng District(City seat),0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Dongcheng District(City seat),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Dongcheng District(City seat),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,Dongcheng District(City seat),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
11,Dongcheng District(City seat),0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.


In [15]:
city_onehot.shape

(169, 36)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [16]:
city_grouped = city_onehot.groupby('Neighborhood').mean().reset_index()
city_grouped.head()

,Neighborhood,American Restaurant,Asian Restaurant,Australian Restaurant,Beijing Restaurant,Cantonese Restaurant,Chinese Aristocrat Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,...,Restaurant,Scandinavian Restaurant,South American Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Xinjiang Restaurant,Yunnan Restaurant,Zhejiang Restaurant
0,Chaoyang District,0.028571,0.057143,0.0,0.00,0.028571,0.00,0.200000,0.00,0.057143,...,0.028571,0.000000,0.000000,0.028571,0.028571,0.000000,0.000000,0.00,0.028571,0.000000
1,Dongcheng District(City seat),0.000000,0.035714,0.0,0.00,0.000000,0.00,0.107143,0.00,0.107143,...,0.000000,0.035714,0.035714,0.035714,0.000000,0.035714,0.035714,0.00,0.071429,0.035714
2,Fangshan District,0.000000,0.000000,0.0,0.00,0.000000,0.00,0.333333,0.00,0.000000,...,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
3,Fengtai District,0.000000,0.000000,0.0,0.00,0.000000,0.00,0.333333,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
4,Haidian District,0.000000,0.000000,0.0,0.02,0.000000,0.02,0.260000,0.04,0.020000,...,0.020000,0.000000,0.000000,0.020000,0.020000,0.040000,0.000000,0.06,0.000000,0.000000


#### Let's confirm the new size

In [17]:
city_grouped.shape

(13, 36)

#### Let's print each neighborhood along with the top 5 most common venues

In [18]:
num_top_venues = 5

for hood in city_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = city_grouped[city_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Chaoyang District----
                     venue  freq
0      Japanese Restaurant  0.26
1       Chinese Restaurant  0.20
2        Hotpot Restaurant  0.06
3  New American Restaurant  0.06
4      Dumpling Restaurant  0.06


----Dongcheng District(City seat)----
                 venue  freq
0   Mexican Restaurant  0.11
1   Chinese Restaurant  0.11
2  Dumpling Restaurant  0.11
3          Pizza Place  0.11
4    Yunnan Restaurant  0.07


----Fangshan District----
                       venue  freq
0       Fast Food Restaurant  0.33
1         Chinese Restaurant  0.33
2                 Restaurant  0.33
3  South American Restaurant  0.00
4  Middle Eastern Restaurant  0.00


----Fengtai District----
                       venue  freq
0       Fast Food Restaurant  0.67
1         Chinese Restaurant  0.33
2  South American Restaurant  0.00
3  Middle Eastern Restaurant  0.00
4    New American Restaurant  0.00


----Haidian District----
                  venue  freq
0    Chinese Restaurant  0.26


#### Let's put that into a _pandas_ dataframe


First, let's write a function to sort the venues in descending order.


In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = city_grouped['Neighborhood']

for ind in np.arange(city_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(city_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Chaoyang District,Japanese Restaurant,Chinese Restaurant,New American Restaurant,Asian Restaurant,Dumpling Restaurant,Hotpot Restaurant,Italian Restaurant,Japanese Curry Restaurant,Cantonese Restaurant,French Restaurant
1,Dongcheng District(City seat),Pizza Place,Dumpling Restaurant,Chinese Restaurant,Mexican Restaurant,Peking Duck Restaurant,Italian Restaurant,Yunnan Restaurant,Hubei Restaurant,Japanese Restaurant,French Restaurant
2,Fangshan District,Restaurant,Chinese Restaurant,Fast Food Restaurant,Dumpling Restaurant,Hubei Restaurant,Hotpot Restaurant,Halal Restaurant,German Restaurant,French Restaurant,Zhejiang Restaurant
3,Fengtai District,Fast Food Restaurant,Chinese Restaurant,Zhejiang Restaurant,Dumpling Restaurant,Hubei Restaurant,Hotpot Restaurant,Halal Restaurant,German Restaurant,French Restaurant,Dim Sum Restaurant
4,Haidian District,Chinese Restaurant,Pizza Place,Korean Restaurant,Fast Food Restaurant,Xinjiang Restaurant,Hotpot Restaurant,Vegetarian / Vegan Restaurant,Dim Sum Restaurant,Indian Restaurant,Middle Eastern Restaurant


## 4. Cluster Neighborhoods

Run _k_-means to cluster the neighborhood into 2 clusters.


In [21]:
# set number of clusters
kclusters = 2

city_grouped_clustering = city_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [22]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

city_merged = neighborhoods

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
city_merged = city_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Borough')

In [23]:
city_merged = city_merged.drop('Unnamed: 0', 1) # check the last columns!
city_merged = city_merged[city_merged['Cluster Labels'].notna()]
city_merged['Cluster Labels'] = city_merged['Cluster Labels'].astype(int)
city_merged['Cluster Labels'].values # check the last columns!

array([0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1])

Finally, let's visualize the resulting clusters


In [24]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(city_merged['Latitude'], city_merged['Longitude'], city_merged['Borough'], city_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters


Examining each cluster.

#### Cluster 1


In [25]:
city_merged.loc[city_merged['Cluster Labels'] == 0, city_merged.columns[[1] + list(range(5, city_merged.shape[1]))]]

,Chinese,Area (km²),Population (2010 census)[2],Density (/km²),Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,东城区,40.6,919000,22635,39.93264,116.41834,0,Pizza Place,Dumpling Restaurant,Chinese Restaurant,Mexican Restaurant,Peking Duck Restaurant,Italian Restaurant,Yunnan Restaurant,Hubei Restaurant,Japanese Restaurant,French Restaurant
1,西城区,46.5,1243000,26731,39.93317,116.37337,0,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Asian Restaurant,Yunnan Restaurant,Japanese Restaurant,Zhejiang Restaurant,New American Restaurant,Dumpling Restaurant,French Restaurant
2,朝阳区,470.8,3545000,7530,39.95462,116.50290,0,Japanese Restaurant,Chinese Restaurant,New American Restaurant,Asian Restaurant,Dumpling Restaurant,Hotpot Restaurant,Italian Restaurant,Japanese Curry Restaurant,Cantonese Restaurant,French Restaurant
5,海淀区,426.0,3281000,7702,39.99064,116.28868,0,Chinese Restaurant,Pizza Place,Korean Restaurant,Fast Food Restaurant,Xinjiang Restaurant,Hotpot Restaurant,Vegetarian / Vegan Restaurant,Dim Sum Restaurant,Indian Restaurant,Middle Eastern Restaurant
7,房山区,1866.7,945000,506,39.69833,115.99250,0,Restaurant,Chinese Restaurant,Fast Food Restaurant,Dumpling Restaurant,Hubei Restaurant,Hotpot Restaurant,Halal Restaurant,German Restaurant,French Restaurant,Zhejiang Restaurant
9,顺义区,980.0,877000,895,40.12175,116.64783,0,Italian Restaurant,Zhejiang Restaurant,Dumpling Restaurant,Hubei Restaurant,Hotpot Restaurant,Halal Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant,Dim Sum Restaurant
13,平谷区,1075.0,416000,387,40.14554,117.10948,0,Fast Food Restaurant,Australian Restaurant,Middle Eastern Restaurant,Chinese Restaurant,Zhejiang Restaurant,Hubei Restaurant,Hotpot Restaurant,Halal Restaurant,German Restaurant,French Restaurant


#### Cluster 2


In [26]:
city_merged.loc[city_merged['Cluster Labels'] == 1, city_merged.columns[[1] + list(range(5, city_merged.shape[1]))]]

,Chinese,Area (km²),Population (2010 census)[2],Density (/km²),Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,丰台区,304.2,2112000,6943,39.84757,116.27227,1,Fast Food Restaurant,Chinese Restaurant,Zhejiang Restaurant,Dumpling Restaurant,Hubei Restaurant,Hotpot Restaurant,Halal Restaurant,German Restaurant,French Restaurant,Dim Sum Restaurant
4,石景山区,89.8,616000,6860,39.91124,116.18766,1,Fast Food Restaurant,Pizza Place,Zhejiang Restaurant,Italian Restaurant,Hubei Restaurant,Hotpot Restaurant,Halal Restaurant,German Restaurant,French Restaurant,Dumpling Restaurant
6,门头沟区,1331.3,290000,218,39.93819,116.09307,1,Fast Food Restaurant,Zhejiang Restaurant,Dumpling Restaurant,Hubei Restaurant,Hotpot Restaurant,Halal Restaurant,German Restaurant,French Restaurant,Dim Sum Restaurant,Italian Restaurant
8,通州区,870.0,1184000,1361,39.90395,116.66183,1,Fast Food Restaurant,Chinese Restaurant,Zhejiang Restaurant,Dumpling Restaurant,Hubei Restaurant,Hotpot Restaurant,Halal Restaurant,German Restaurant,French Restaurant,Dim Sum Restaurant
12,怀柔区,2557.3,373000,146,40.32353,116.63076,1,Fast Food Restaurant,Zhejiang Restaurant,Dumpling Restaurant,Hubei Restaurant,Hotpot Restaurant,Halal Restaurant,German Restaurant,French Restaurant,Dim Sum Restaurant,Italian Restaurant
14,密云区,2335.6,468000,200,40.37587,116.84191,1,Fast Food Restaurant,Zhejiang Restaurant,Dumpling Restaurant,Hubei Restaurant,Hotpot Restaurant,Halal Restaurant,German Restaurant,French Restaurant,Dim Sum Restaurant,Italian Restaurant
